**Sustainable Software Development, block course, March 2021**  
*Scientific Software Center, Institute for Scientific Computing, Dr. Inga Ulusoy*

# Analysis of the data

Imagine you perform a "measurement" of some type and obtain "scientific data". You know what your data represents, but you have only a vague idea how different features in the data are connected, and what information you can extract from the data.

You would start first with going through the data, making sure your data set is complete and that the result is reasonable. Imagine this already happened.

In the next step, you would inspect your data more closely and try to identify structures. That is the step that we are focusing on in this unit.

In the `data` folder, you will find several data files (`*.t` and `*.dat`). These are data files generated through some "new approach" that hasn't been used in your lab before. No previous analysis software exists, and you are going to establish a protocol for this "new approach" and "publish your results".

The data can be grouped into two categories: 
1. data to be analyzed using statistical methods;
2. data to be analyzed using numerical methods.

In your hypothetical lab, you are an "expert" in one particular "method", and your co-worker is an "expert" in the other. Combined these two methods will lead to much more impactful results than if only one of you analyzed the data. Now, the task in this course is to be solved collaboratively with your team member working on one of the analysis approaches, and you working on the other. You will both implement functionality into the same piece of "software", but do so collaboratively through git.

As you do not know yet which analysis is most meaningful for your data, and how to implement it, you will start with a jupyter notebook. You and your team member will work on the same notebook that will be part of a github repository for your project. This is the task for today. Discuss with your team members who will work on the statistical and who on the numerical analysis.

## Step 1

Generate a github repository with the relevant files.

## Step 2

Clone the repository to your local machine.

## Step 3

Start working on task 1 for your analysis approach. 

## Step 4

Create your own branch of the repository and commit your changes to your branch; push to the remote repository.

## Step 5

Open a `pull request` so your team member can review your implementation. Likewise, your team member will ask you to review theirs.

## Step 6

Merge the changes in your branch into `main`. Resolve conflicts.

## Step 7

Repeat working on task; committing and pushing to your previously generated branch or a new branch; open a pull request; merge with main; until you have finished all the tasks in your analysis approach. Delete obsolete branches.

# Start of the analysis notebook

**Author : Massimiliano Christ, Anton Eberhardt**  
*Date : 01.03.2022 
*Affiliation : University of Heidelberg  

Place the required modules in the top, followed by required constants and global functions.

In [ ]:
# required modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sn

In [ ]:
# params for default matplotlib style
mpl.rcParams["figure.figsize"] = [12.0, 8.0]
mpl.rcParams["figure.dpi"] = 200
mpl.rcParams["savefig.dpi"] = 300

mpl.rcParams["font.size"] = 12
mpl.rcParams["legend.fontsize"] = "large"
mpl.rcParams["figure.titlesize"] = "medium"

In [ ]:
# reading of the data files
dfTable = pd.read_table(
    "../data/table.dat", sep=" ", engine="python", skipinitialspace=True
)
dfEfield = pd.read_table(
    "../data/efield.t", sep=" ", engine="python", skipinitialspace=True
)
dfExpec = pd.read_table(
    "../data/expec.t", sep=" ", engine="python", skipinitialspace=True
)
dfNpop = pd.read_table(
    "../data/npop.t", sep=" ", engine="python", skipinitialspace=True
)
dfNstate_i = pd.read_table(
    "../data/nstate_i.t", sep=" ", engine="python", skipinitialspace=True
)

# Statistical analysis

Find correlations in the data sets. Analyse the data statistically and plot your results.  

Here we would want to do everything with pandas and leave the data in a dataframe. The files that are relevant to you are `expect.t`, `npop.t` and `table.dat`.

### Task 1: Read in expec.t and plot relevant data

In [ ]:
# read and plot expec.panda
dfExpec.plot(
    x="time",
    subplots=True,
    figsize=(12, 15),
    ylabel="[A.U.]",
    title=[
        "Norm against time",
        "<x> against time",
        "<y> against time",
        "<z> against time",
        "H field against time",
    ],
)

We can discard the entries norm, \<x>, and \<y> as these are mostly constant.

In [ ]:
# eliminate columns based on the variance: if the variance of the values
# in a column is below a given threshold, that column is discarded
def eliminate_columns_variance(dataFrame, threshold):
    """deletes columns in a given data frame where the variance of the columns is below a
    given threshold"""
    output = dataFrame.copy()
    for columnName in output:
        variance = np.var(output[columnName])
        if variance < threshold:
            del output[columnName]
    return output

In [ ]:
cleanedExpecDF = eliminate_columns_variance(dfExpec, 1e-20)
cleanedNpopDF = eliminate_columns_variance(dfNpop, 1e-5)
cleanedTableDF = eliminate_columns_variance(dfTable, 1e-10)

### Task 2: Create plots of the relevant data and save as .pdf.

In [ ]:
# create plots
plt.figure(figsize=(12, 14))
plt.subplot(2, 1, 1)
plt.plot(cleanedExpecDF["time"], cleanedExpecDF["<z>"])
plt.title("Plotting the z-axis against time")
plt.xlabel("time [s]", fontsize=12)
plt.ylabel("<z> [A.U.]", fontsize=12)
plt.legend()
plt.savefig("z-axis_expected.pdf", format="pdf")

plt.subplot(2, 1, 2)
plt.plot(cleanedExpecDF["time"], cleanedExpecDF["<H>"])
plt.title("Plotting the <H> against time")
plt.xlabel("time [s]", fontsize=12)
plt.ylabel("<H> [A.U.]", fontsize=12)
plt.legend()
plt.savefig("z-axis_expected.pdf", format="pdf", dpi=300)

### Task 3: Read in file `npop.t` and analyze correlations in the data

In [ ]:
# plotting with seaborn
sn.lineplot(x="time", y="value", hue="variable", data=pd.melt(cleanedNpopDF, ["time"]))
plt.title("Plotting the Npop table")
plt.xlabel("time [s]")
plt.ylabel("value [A.U.]")
plt.show()

Plot the remaining columns. Seaborn prefers "long format" (one column for all measurement values, one column to indicate the type) as input, whereas the cvs is in "wide format" (one column per measurement type).

## Quantify the pairwise correlation in the data

- negative correlation: y values decrease for increasing x - large values of one feature correspond to small values of the other feature
- weak or no correlation: no trend observable, association between two features is hardly observable
- positive correlation: y values increase for decreasing x - small values of one feature correspond to small values of the other feature

Remember that correlation does not indicate causation - the reason that two features are associated can lie in their dependence on same factors.

Correlate the value pairs using Pearson's $r$. Pearson's $r$ is a measure of the linear relationship between features:

$r = \frac{\sum_i(x_i − \bar{x})(y_i − \bar{y})}{\sqrt{\sum_i(x_i − \bar{x})^2 \sum_i(y_i − \bar{y})^2}}$

Here, $\bar{x}$ and $\bar{y}$ indicate mean values. $i$ runs over the whole data set. For a positive correlation, $r$ is positive, and negative for a negative correlation, with minimum and maximum values of -1 and 1, indicating a perfectly linear relationship. Weakly or not correlated features are characterized by $r$-values close to 0.

Other measures of correlation that can be used are Spearman's rank (value pairs follow monotonic function) or Kendall's $\tau$ (measures ordinal association), but they do not apply here. You can also define measures yourself.

In [ ]:
# print the correlation matrix

The diagonal values tell us that each value is perfectly correlated with itself. We are not interested in the diagonal values and also not in the correlation with time. We also need to get rid of redundant entries. Finally, we need to find the value pairs that exhibit the highest linear correlation. We still want to know if it is positive or negative correlation, so we cannot get rid of the sign.

In [ ]:
# get rid of time column, lower triangular and diagonal entries of the correlation matrix
# sort the remaing values according to their absolute value, but keep the sign

Note that the entries in the left column are not repeated if they do not change from the row above (so the fourth feature pair is MO3 and MO6).

### Task 4: Print the resulting data to a file

In [ ]:
# write to file

### Task 5: Calculate the Euclidean distance (L2 norm) for the vectors in `table.dat`


The Euclidean distance measures the distance between to objects that are not points:

$d(p,q) = \sqrt{\left(p-q\right)^2}$

In this case, consider each of the columns in table.dat as a vector in Euclidean space, where column $r(x)$ and column $v(x)$ denote a pair of vectors that should be compared, as well as $r(y)$ and $v(y)$, and r(z) and v(z).

(Background: These are dipole moment components in different gauges, the length and velocity gauge.)

In [ ]:
# read in table.dat - I suggest reading it as a numpy array
# replace the NaNs by zero

Now calculate how different the vectors in column 2 are from column 3, column 4 from column 5, and column 6 from column 7.

In [ ]:
# calculate the Euclidean distance

In [ ]:
# plot the result and save to a .pdf

In [ ]:
# print the result to a file

# Numerical analysis

Analyze the data using autocorrelation functions and discrete Fourier transforms. Plot your results.

In [ ]:
# define some global functions

### Task 1: Read in `efield.t` and Fourier-transform relevant columns

In [ ]:
def df_filter_constant(Dataframe, Ignore, VarThreshold=0):
    """Find and remove columns which do not have a variance bigger
    than the given threshold.
    :param Dataframe: Dataframe to be scanned fro constant columns.
    :type Dataframe: pandas Dataframe

    :param Ignore: List of columns to be ignored,
    columns will end up in returned dataframe regardless of variance.
    :type Ignore: array-like

    :param Varthreshold: All columns with less or equal this variance will not
    end up in returned data frame. Default is 0, so only constant columns will be discarded
    :type Varthreshold: float

    :raises KeyError: Raises error, when Ignore contains keywords that Dataframe does not contain

    :return: Copy of Dataframe, where all columns with variance less than given treshold are discarded
    :rtype: pandas.Dataframe
    """

    NewDf = Dataframe

    for key in Ignore:
        try:
            NewDf[key]

        except KeyError as e:
            print(e, "Column to be ignored not found in dataframe")

    for key in Dataframe.columns.values:

        if key in Ignore:
            continue

        if np.var(NewDf[key]) <= VarThreshold:
            NewDf = NewDf.drop(key, axis=1)

    return NewDf


def plot_relevant_columns(Dataframe, xAxis, figsize=(14, 10), VarTreshold=0):

    """Plot columns with Variance bigger than the given threshold.

    :param Dataframe: Dataframe to be plotted.
    :type Dataframe: pandas Dataframe

    :param xAxis: column to be used for the xAxis of the plots
    :type xAxis: Key for dataframe (most likely string)

    :param Varthreshold: All columns with less or equal this variance will not
    end up in returned data frame. Default is 0, so only constant columns will be discarded
    :type Varthreshold: float

    :param figsize: Size of figure
    :type figsize: tuple

    :return: None
    :rtype: None
    """
    try:
        NewDF = df_filter_constant(Dataframe, [xAxis], VarThreshold=VarTreshold)
    except KeyError as e:
        print(e, "Specified x-axis not found in dataframe")

    NewDF.plot(x=xAxis, subplots=True, figsize=figsize)

In [ ]:
data = {"time": np.arange(5), "one": np.ones(5), "zero": np.zeros(5)}

testDf = pd.DataFrame(data)

print(df_filter_constant(testDf, ["zero"], VarThreshold=0))

In [ ]:
# read and plot efield.t

test = df_filter_constant(dfEfield, ["time"], VarThreshold=0.0001)

plot_relevant_columns(dfEfield, "time", figsize=(14, 10), VarTreshold=0)

Only y-values have any variance, so we fourier-transform/plot only this.

In [ ]:
efield_y_fft = np.fft.fft(dfEfield["y"])

plt.figure(figsize=(14, 7))

plt.subplot(121)
plt.plot(dfEfield["time"], dfEfield["y"])
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")

plt.subplot(122)
plt.scatter(dfEfield["time"], np.real(efield_y_fft))
plt.xlabel("Frequencies [Hz]")

plt.show()

Here we are interested in column 2 since the others are constant.

In [ ]:
# discrete Fourier transform of the remaining column: You only need the real frequencies

### Task 2: Generate a plot of your results to be saved as pdf.

In [ ]:
# plot your results

### Task 3: Calculate the autocorrelation function from nstate_i.t
The autocorrelation function measures how correlated subsequent vectors are with an initial vector; ie. 

$\Psi_{corr} = \langle \Psi(t=0) | \Psi(t) \rangle = \int_0^{tfin} \Psi(0)^* \Psi(t) dt$

Since we are in a numerical representation, the integral can be replaced with a sum; and the given vectors are already normalized.

In [ ]:
# read in as numpy array

dfNstate_i = pd.read_table(
    "../data/nstate_i.t",
    sep="   ",
    engine="python",
    skipinitialspace=True,
    skiprows=1,
    header=None,
)

newDf = pd.DataFrame([])

for key in dfNstate_i.columns.values:

    if key % 2 == 0:  # skip first row and all imaginary rows
        continue

    newCol = dfNstate_i[key].to_numpy() + 1j * dfNstate_i[key + 1].to_numpy()

    if key == 1:
        newDf = pd.DataFrame({"Vectorcomponent " + str(int((key - 1) / 2)): newCol})

    else:
        newDf["Vectorcomponent " + str(int((key - 1) / 2))] = newCol

newDf["time"] = dfNstate_i[0]


def autocorrelation(Dataframe, Ignore):

    for key in Ignore:
        newDf = Dataframe.drop(key, axis=1)

    vec0 = newDf.loc[0, :].to_numpy()

    result = np.zeros(newDf.shape, dtype=complex)

    for i in range(newDf.shape[0]):

        j = 0

        while j <= i:
            result[i, :] += np.conj(vec0) * newDf.loc[j, :].to_numpy()
            j += 1

    return result


final_result = autocorrelation(newDf, ["time"])

In [ ]:
# correct the data representation: this is in fact a complex matrix
# the real part of each matrix column is contained in numpy array column 0, 2, 4, 6, ...
# the imaginary part of each matrix column is contained in numpy array column 1, 3, 5, 7, ...
# convert the array that was read as dtype=float into a dtype=complex array

In [ ]:
# for the autocorrelation function, we want the overlap between the first vector at time 0 and all
# subsequent vectors at later times - the sum of the product of initial and subsequent vectors for each time step

### Task 4: Generate a plot of your results to be saved as pdf.

In [ ]:
# plot the autocorrelation function - real, imaginary and absolute part

### Task 5: Discrete Fourier transform of the autocorrelation function

In [ ]:
# discrete Fourier-transform the autocorrelation function - now we need all frequency components,
# also the negative ones

### Task 6: Generate a plot of your results to be saved as pdf.

In [ ]:
# plot the power spectrum (abs**2)